In [65]:
#Creating wikipedia tool
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

api_wraper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wraper)
wiki.name

'wikipedia'

In [66]:
#Creating Arxiv tool
from langchain.tools import ArxivQueryRun
from langchain.utilities import ArxivAPIWrapper

arx_wraper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arx_wraper)
arxiv.name

'arxiv'

In [67]:
#creating webbase tool
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
loader = WebBaseLoader("https://docs.smith.langchain.com/")
doc = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
spli_doc = splitter.split_documents(doc)
db = FAISS.from_documents(spli_doc,HuggingFaceEmbeddings())
db_retriver = db.as_retriever()

In [68]:
#Creating retriever tool
from langchain.tools.retriever import create_retriever_tool

web_tool = create_retriever_tool(db_retriver,'web tool','this tool is used to retrive information from langchain')
web_tool.name

'web tool'

In [69]:
#creatin tool for pdf loader
from langchain.document_loaders import PyPDFLoader
loader_pdf = PyPDFLoader('attention.pdf')
doc = loader_pdf.load()
split = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
doc_split = split.split_documents(doc)
vec_db = FAISS.from_documents(doc_split,HuggingFaceEmbeddings())
retriver = vec_db.as_retriever()

In [70]:
#creating pdf retriver tool
pdf_tool= create_retriever_tool(retriver,'pdf tool', 'This tool will retrieve information from pdf.')
pdf_tool.name

'pdf tool'

In [71]:
tools = [wiki,arxiv,web_tool,pdf_tool]

In [72]:
#Designing the prompt
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [73]:
from dotenv import load_dotenv
import os

load_dotenv()


os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 5


In [74]:
#loading the model

from langchain_groq import ChatGroq


llm = ChatGroq(model='llama3-8b-8192')

In [75]:
#creating the agent tooling
from langchain.agents import create_tool_calling_agent

agents = create_tool_calling_agent(tools=tools,prompt=prompt,llm=llm)

In [76]:
from langchain.agents import AgentExecutor

agent_ex = AgentExecutor(tools=tools,agent=agents,verbose=1)

In [77]:
agent_ex.invoke({'input':'what is attention'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'attention'}`


Page: Attention
Summary: Attention or focus, is the concentration of awareness on some phenomenon to the exclusion of other stimuli. It is the selective concentration on discrete information, either s
Invoking: `wikipedia` with `{'query': 'attention neuroscience'}`


Page: Attention
Summary: Attention or focus, is the concentration of awareness on some phenomenon to the exclusion of other stimuli. It is the selective concentration on discrete information, either sIt seems like the tool call yielded the same information as the previous one. Let's try to provide a direct response instead.

Attention is the ability to focus on a specific stimulus or task while ignoring others. It's a fundamental aspect of human cognition that allows us to filter out distractions and concentrate on what's important. In neuroscience, attention is a complex process that involves multiple brain regions and networks,

{'input': 'what is attention',
 'output': "It seems like the tool call yielded the same information as the previous one. Let's try to provide a direct response instead.\n\nAttention is the ability to focus on a specific stimulus or task while ignoring others. It's a fundamental aspect of human cognition that allows us to filter out distractions and concentrate on what's important. In neuroscience, attention is a complex process that involves multiple brain regions and networks, including the prefrontal cortex, parietal cortex, and anterior cingulate cortex. There are different types of attention, including focused attention, divided attention, and selective attention, each with its own unique characteristics and functions."}